### 

In [3]:
from typing import NamedTuple

In [4]:
class Position(NamedTuple):
    x: int
    y: int

In [5]:
class GameAction(Enum):
    """Base class for game actions"""
    @classmethod
    @abstractmethod
    def get_all_actions(cls) -> List["GameAction"]:
        """Returns all possible actions for the game"""
        raise NotImplementedError

    def __repr__(self):
        return self.name.lower()
    
    @classmethod
    @abstractmethod
    def default_action(cls) -> "GameAction":
        """Returns the default action for the game"""
        raise NotImplementedError

class TextAdventureGameAction(GameAction):
    UP = "up"
    DOWN = "down"
    LEFT = "left"
    RIGHT = "right"

    @classmethod
    def get_all_actions(cls) -> List["TextAdventureGameAction"]:
        return [cls.UP, cls.DOWN, cls.LEFT, cls.RIGHT]
    
    def __repr__(self):
        return self.name.lower()

In [6]:
class TextAdventureGame:
    def __init__(self, map_size: int = 8):
        self.map = [
            ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o'],
            ['o', 'w', 'w', 'w', 'w', 'w', 'w', 'o'],
            ['o', 'w', 'w', 'w', 'w', 'w', 'w', 'o'],
            ['o', 'w', 'w', 'w', 'w', 'w', 'w', 'o'],
            ['o', 'w', 'w', 'w', 'w', 'w', 'w', 'o'],
            ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'o']
        ]
        self.current_position = Position(1, 1)
        self.previous_position = Position(1, 1)

    def is_valid_move(self, x: int, y: int) -> bool:
        if (0 <= y < len(self.map) and 
            0 <= x < len(self.map[0]) and 
            self.map[y][x] == 'w'):
            return True
        return False

    def update(self, action: TextAdventureGameAction) -> bool:
        new_x, new_y = self.current_position.x, self.current_position.y
        
        if action == TextAdventureGameAction.UP:
            new_y -= 1
        elif action == TextAdventureGameAction.DOWN:
            new_y += 1
        elif action == TextAdventureGameAction.LEFT:
            new_x -= 1
        elif action == TextAdventureGameAction.RIGHT:
            new_x += 1

        valid = self.is_valid_move(new_x, new_y)
        if valid:
            self.previous_position = self.current_position
            self.current_position = Position(new_x, new_y)

        return valid

    def get_map_string(self) -> str:
        self.map[self.previous_position.y][self.previous_position.x] = "w"
        self.map[self.current_position.y][self.current_position.x] = "p"
        return '\n'.join([''.join(row) for row in self.map])